In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
import zipfile
zip_path = '/content/drive/MyDrive/sign_lang/asl_alphabet_train.zip'
extract_to = '/content/q'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

In [ ]:
# Define data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 80% training, 20% validation
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    '/content/q/asl_alphabet_train',
    target_size=(224, 224),  # EfficientNetB0 default input size
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/content/q/asl_alphabet_train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.


In [ ]:
# Load the EfficientNetB0 model pre-trained on ImageNet
base_model = EfficientNetB0(weights='imagenet', include_top=False)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(29, activation='softmax')(x)

In [ ]:
# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Define callbacks
checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator,
    callbacks=[checkpoint, early_stopping]
)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1108s 497ms/step - accuracy: 0.0360 - loss: 3.3698 - val_accuracy: 0.0345 - val_loss: 3.3674
Epoch 2/5
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1026s 471ms/step - accuracy: 0.0336 - loss: 3.3676 - val_accuracy: 0.0345 - val_loss: 3.3673
Epoch 3/5
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1018s 460ms/step - accuracy: 0.0347 - loss: 3.3675 - val_accuracy: 0.0345 - val_loss: 3.3674
Epoch 4/5
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1054s 465ms/step - accuracy: 0.0341 - loss: 3.3676 - val_accuracy: 0.0345 - val_loss: 3.3674
Epoch 5/5
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1004s 460ms/step - accuracy: 0.0326 - loss: 3.3675 - val_accuracy: 0.0345 - val_loss: 3.3674


In [ ]:
# Unfreeze and fine-tune
for layer in base_model.layers:
    layer.trainable = True

In [ ]:
import json
# Save history object after initial training
with open('/content/drive/MyDrive/sign_lang/training_history.json', 'w') as f:
    json.dump(history.history, f)

In [ ]:
import json
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model

# Mount Google Drive (if using Google Colab)
from google.colab import drive
drive.mount('/content/drive')

# Load the saved model
model = load_model('/content/drive/MyDrive/sign_lang/best_model.keras')

# Set up the data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 80% training, 20% validation
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    '/content/q/asl_alphabet_train',
    target_size=(224, 224),  # EfficientNetB0 default input size
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/content/q/asl_alphabet_train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Continue training or fine-tuning
history_fine = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator,
    callbacks=[checkpoint, early_stopping]
)

# Save the fine-tuning history
with open('/content/drive/MyDrive/sign_lang/fine_tuning_history.json', 'w') as f:
    json.dump(history_fine.history, f)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1070s 479ms/step - accuracy: 0.0351 - loss: 3.3681 - val_accuracy: 0.0345 - val_loss: 3.3674
Epoch 2/5
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1031s 473ms/step - accuracy: 0.0351 - loss: 3.3675 - val_accuracy: 0.0345 - val_loss: 3.3673
Epoch 3/5
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1009s 463ms/step - accuracy: 0.0342 - loss: 3.3675 - val_accuracy: 0.0345 - val_loss: 3.3674
Epoch 4/5
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1006s 462ms/step - accuracy: 0.0341 - loss: 3.3674 - val_accuracy: 0.0345 - val_loss: 3.3673
Epoch 5/5
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 1076s 477ms/step - accuracy: 0.0348 - loss: 3.3674 - val_accuracy: 0.0345 - val_loss: 3.3673
